In [1]:
from oumi.core.analyze.llm_judge_analyzer import LLMJudgeAnalyzer

print("Available preset prompts:", LLMJudgeAnalyzer.list_presets())

Available preset prompts: ['instruction_quality', 'response_quality', 'conversation_coherence', 'safety', 'helpfulness', 'factuality']


In [2]:
import os
import sys


# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [19]:
from oumi.core.configs import AnalyzeConfig, SampleAnalyzerParams

llm_conversation_coherence_params = SampleAnalyzerParams(
    id="llm_judge",
    params={
        # Use a preset prompt (available: instruction_quality, response_quality,
        # conversation_coherence, safety, helpfulness, factuality)
        "prompt_preset": "conversation_coherence",
        # Inference configuration
        "inference_config": {
            "model_name": "gpt-4o-mini",  # or "gpt-4o", "claude-3-5-sonnet-20241022", etc.
            "engine": "openai",  # or "vllm", "native" for local models
            "temperature": 0.1,  # Low temperature for consistent judgments
            "max_tokens": 256,
            # For OpenAI, API key is read from OPENAI_API_KEY env var by default
            # For Anthropic, use ANTHROPIC_API_KEY env var
            # Or specify explicitly:
            # "api_key_env": "OPENAI_API_KEY",  # or "ANTHROPIC_API_KEY"
        },
        "batch_size": 10,  # Process 10 samples at a time
        "max_text_length": 4000,  # Truncate long texts
        "parse_json_response": True,  # Parse JSON from LLM response
    },
)


In [20]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Create the full AnalyzeConfig
config = AnalyzeConfig(
    # Dataset configuration
    dataset_path="/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl",
    # Analysis configuration
    sample_count=5,  # Analyze first 100 samples (LLM calls can be expensive!)
    analyzers=[
        llm_conversation_coherence_params,
    ],  # Just the LLM judge
    # Output configuration
    output_path="./analysis_output/banking77_llm_judge",
    generate_report=False,  # Optional: generate HTML report
)

# Finalize and run
config.finalize_and_validate()

from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

analyzer = DatasetAnalyzer(config)
analyzer.analyze_dataset()


[2025-12-30 16:45:26,779][oumi][rank0][pid:67279][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2025-12-30 16:45:26,780][oumi.utils.analysis_utils][rank0][pid:67279][MainThread][INFO]][analysis_utils.py:225] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2025-12-30 16:45:26,780][oumi][rank0][pid:67279][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2025-12-30 16:45:26,780][oumi][rank0][pid:67279][MainThread][INFO]][dataset_analyzer.py:304] Initialized sample analyzer: llm_judge
[2025-12-30 16:45:26,803][oumi][rank0][pid:67279][MainThread][INFO]][dataset_analyzer.py:347] Starting analysis of dataset: None
[2025-12-30 16:45:26,804][oumi][rank0][pid:67279][MainThread][INFO]][dataset_analyzer.py:348] Using 1 sample analyzers: ['llm_judge']
[2025-12-30 16:45:26,804][oumi][rank0][pid:67279][MainThrea

Converting Unknown Dataset to DataFrames: 100%|██████████| 5/5 [00:00<00:00, 1628.22item/s]


[2025-12-30 16:45:26,822][oumi][rank0][pid:67279][MainThread][INFO]][llm_judge_analyzer.py:444] Initialized LLM Judge with model: gpt-4o-mini, engine: openai


100%|██████████| 5/5 [00:08<00:00,  1.66s/it]


[2025-12-30 16:45:35,111][oumi][rank0][pid:67279][MainThread][INFO]][llm_judge_analyzer.py:780] Skipping message-level analysis (analyze_message_level=False). Set analyze_message_level=True to enable.


In [21]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 12
len(merged_columns): 12


In [6]:
analyzer_names = [a.id for a in config.analyzers]
analyzer_names

['llm_judge']

# Conv level

In [22]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[0]


filtered_cols = filter_analyzer_columns(conv_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])
    # print("\nInput:")
    # print(f"source_column: {info.source_column}")
    # print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


Analyzer: llm_judge
metric: score
description: LLM judge score (0-10, higher = better quality)
value: 10.0


metric: label
description: LLM judge label/category for the sample
value: coherent


metric: reasoning
description: LLM judge reasoning/explanation
value: The conversation maintains a logical flow, references previous context appropriately, and shows consistent topic threading without any repetition or contradictions.


metric: raw_response
description: Raw LLM response before parsing
value: {
  "score": 10,
  "label": "coherent",
  "reasoning": "The conversation maintains a logical flow, references previous context appropriately, and shows consistent topic threading without any repetition or contradictions."
}




# Message level

In [9]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

msg_columns = analyzer.message_df.columns
row = analyzer.message_df.iloc[5]

# Choose the analzyer to analyze
analyzer_name = analyzer_names[0]


filtered_cols = filter_analyzer_columns(msg_columns, analyzer_id=analyzer_name)
if filtered_cols:
    print(f"Analyzer: {analyzer_name}")
    info = parse_analyzer_column_name(filtered_cols[0])

    print("\nInput:")
    print(f"[{row['role']}]")
    # print(f"{row[info.source_column]}\n")
    # print(f"source_column: {info.source_column}")
    # print(f"{row[info.source_column]}\n")

    for col in filtered_cols:
        info = parse_analyzer_column_name(col)
        print(f"metric: {info.metric_name}")
        # print(f"type: {schema[col]['type']}")
        # print(f"content_type: {schema[col]['content_type']}")
        print(f"description: {schema[col]['description']}")
        print(f"value: {row[col]}")
        print("\n")
else:
    print(f"No columns found for analyzer: {analyzer_name}")


Analyzer: llm_judge

Input:
[assistant]
metric: score
description: LLM judge score (0-10, higher = better quality)
value: 10.0


metric: label
description: LLM judge label/category for the sample
value: excellent


metric: reasoning
description: LLM judge reasoning/explanation
value: The response is a valid numeric ID (69) within the specified range (0-76) and is presented in the correct format without any extra text.


metric: raw_response
description: Raw LLM response before parsing
value: ```json
{
  "score": 10,
  "label": "excellent",
  "reasoning": "The response is a valid numeric ID (69) within the specified range (0-76) and is presented in the correct format without any extra text."
}
```


